In [ ]:
# ==========================
# Install all dependencies
# ==========================
!pip install -q torchaudio omegaconf soundfile
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q groq huggingface-hub kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

# ==========================
# Imports
# ==========================
import os, base64, time
import torch
import whisper
from google.colab import output
from IPython.display import Javascript, display, Audio
from groq import Groq
from kokoro import KPipeline
import soundfile as sf

# ==========================
# Initialize Models
# ==========================
print("🔄 Loading Whisper model...")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
whisper_model = whisper.load_model("medium.en").to(DEVICE)

print("🔄 Loading Groq client...")
groq_client = Groq(api_key="Use your Groq API Key")

print("🔄 Loading Kokoro TTS...")
tts_pipeline = KPipeline(lang_code='a')
voice_choice = 'af_heart'

# Initialize conversation
system_message = {"role": "system", "content": "You are a helpful assistant. Keep your responses concise and conversational."}
conversation_history = [system_message]

# Global state
recording_counter = 0
is_running = True

print("\n✅ All models loaded successfully!")
print("=" * 50)

# ==========================
# Voice Chat Functions
# ==========================

def record_audio(seconds=10):
    """Record audio from microphone"""
    global recording_counter
    recording_counter += 1
    filename = f"/content/voice_{recording_counter}.wav"

    print(f"\n🎤 Recording for {seconds} seconds... Speak now!")

    display(Javascript(f"""
    async function recordAudio(sec) {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const mediaRecorder = new MediaRecorder(stream);
      let chunks = [];

      mediaRecorder.ondataavailable = e => chunks.push(e.data);
      mediaRecorder.start();

      await new Promise(r => setTimeout(r, sec * 1000));
      mediaRecorder.stop();

      await new Promise(r => mediaRecorder.onstop = r);

      const blob = new Blob(chunks, {{ type: "audio/wav" }});
      const reader = new FileReader();

      reader.readAsDataURL(blob);
      await new Promise(r => reader.onload = r);

      google.colab.kernel.invokeFunction(
        "notebook.save_audio",
        [reader.result, "{filename}"],
        {{}}
      );
    }}
    recordAudio({seconds});
    """))

def save_audio(b64_data, filename):
    """Save recorded audio and process it"""
    data = base64.b64decode(b64_data.split(",")[1])
    with open(filename, "wb") as f:
        f.write(data)
    print(f"✅ Recording saved: {filename} ({os.path.getsize(filename)} bytes)")

    # Process the recording
    process_voice_input(filename)

def process_voice_input(filename):
    """Process user voice input and generate AI response"""
    global conversation_history, is_running

    # Transcribe
    print("\n🔄 Transcribing your voice...")
    result = whisper_model.transcribe(filename, verbose=False, language="en")

    user_text = result["text"].strip()

    # Check if recording was empty or too short
    if not user_text or len(user_text) < 3:
        print("\n⚠️ No speech detected. Please try again.")
        time.sleep(1)
        if is_running:
            continue_conversation()
        return

    print(f"\n👤 You said: {user_text}")

    # Check for exit command
    exit_keywords = ["exit", "quit", "stop", "goodbye", "bye"]
    if any(keyword in user_text.lower() for keyword in exit_keywords):
        print("\n👋 Ending conversation. Goodbye!")
        is_running = False

        # Generate goodbye message
        goodbye_text = "Goodbye! It was nice talking to you."
        print(f"\n🤖 Assistant: {goodbye_text}")
        generate_speech(goodbye_text)
        return

    # Add to conversation
    conversation_history.append({"role": "user", "content": user_text})

    # Get LLM response
    print("\n🔄 Getting AI response...")
    try:
        chat_completion = groq_client.chat.completions.create(
            messages=conversation_history,
            model="llama-3.3-70b-versatile",
            max_tokens=150  # Keep responses concise
        )
        assistant_text = chat_completion.choices[0].message.content
        print(f"\n🤖 Assistant: {assistant_text}")

        # Add to conversation
        conversation_history.append({"role": "assistant", "content": assistant_text})

        # Generate and play speech
        generate_speech(assistant_text)

    except Exception as e:
        print(f"\n❌ Error getting AI response: {e}")
        is_running = False
        return

    # Continue conversation automatically
    if is_running:
        time.sleep(0.5)  # Short pause before next recording
        continue_conversation()

def generate_speech(text):
    """Generate and play TTS audio"""
    print("\n🔊 Generating voice response...")

    try:
        generator = tts_pipeline(text, voice=voice_choice)
        total_duration = 0

        for i, (gs, ps, audio) in enumerate(generator):
            audio_file = f'/content/assistant_response_{recording_counter}_{i}.wav'
            sf.write(audio_file, audio, 24000)
            display(Audio(audio_file, rate=24000, autoplay=True))

            # Calculate duration and wait for audio to finish
            duration = len(audio) / 24000
            total_duration += duration
            time.sleep(duration + 0.2)  # Small buffer

        print(f"✅ Voice response complete ({total_duration:.1f}s)")

    except Exception as e:
        print(f"❌ Error generating speech: {e}")

def continue_conversation():
    """Continue the conversation with next recording"""
    global is_running

    if not is_running:
        print("\n✅ Conversation ended.")
        return

    print("\n" + "=" * 50)
    print("🎤 Your turn to speak!")
    print("💡 Say 'exit', 'quit', or 'goodbye' to end the conversation")
    print("=" * 50 + "\n")

    # Start next recording
    record_audio(seconds=10)

# Register callback
output.register_callback("notebook.save_audio", save_audio)

# ==========================
# Start Interactive Voice Chat
# ==========================
print("\n" + "=" * 50)
print("🎙️ INTERACTIVE VOICE CHAT")
print("=" * 50)
print("\n📋 How it works:")
print("1. You speak (10 seconds)")
print("2. AI responds with voice")
print("3. Automatically your turn again")
print("4. Say 'exit' or 'goodbye' to stop")
print("\n" + "=" * 50)
print("🚀 Starting conversation...")
print("=" * 50 + "\n")

# Start first recording


In [ ]:
time.sleep(1)
record_audio(seconds=10)